In [24]:
import os
import gc
import json
import random
import numpy as np
import pandas as pd
import nltk
import torch
import optuna
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, BartForConditionalGeneration,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score
import re
import shutil
import logging
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [2]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
optuna.trial.FixedTrial.seed = SEED
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Check CUDA availability
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("CUDNN version:", torch.backends.cudnn.version())

CUDA available: True
CUDA version: 12.8
CUDNN version: 90701


In [4]:
# NLTK downloads
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
except Exception as e:
    logger.error(f"NLTK download failed: {e}")
    raise

In [5]:
# Define directories
PROJECT_ROOT = r"D:\A_CSE499"
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "output_bloom")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
# Define Bloom levels and corresponding special tokens
BLOOM_LEVELS = {
    'remembering': '<REM>',
    'understanding': '<UND>',
    'applying': '<APP>',
    'analyzing': '<ANA>'
}

In [8]:
# Convert JSON to CSV
def convert_json_to_csv(json_file, csv_file, required_fields):
    if not os.path.exists(json_file):
        logger.error(f"JSON file not found: {json_file}")
        raise FileNotFoundError(f"JSON file not found: {json_file}")
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            logger.error(f"Data in {json_file} must be a list of records")
            raise ValueError(f"Data in {json_file} must be a list of records")
        for record in data:
            if not all(field in record for field in required_fields):
                logger.error(f"Missing required fields in {json_file}: {record}")
                raise ValueError(f"Missing required fields in {json_file}")
        df = pd.DataFrame(data)
        df.to_csv(csv_file, index=False, encoding='utf-8')
        logger.info(f"Converted {json_file} to {csv_file}")
        return df
    except Exception as e:
        logger.error(f"Error converting {json_file} to CSV: {e}")
        raise

# Convert JSON datasets to CSV
required_fields = ['context', 'question', 'level']
eduqg_json = os.path.join(DATA_DIR, "eduqg_903.json")
merged_json = os.path.join(DATA_DIR, "merged_600_per_level.json")
eduqg_csv = os.path.join(DATA_DIR, "eduqg_903.csv")
merged_csv = os.path.join(DATA_DIR, "merged_600_per_level.csv")
eduqg_df = convert_json_to_csv(eduqg_json, eduqg_csv, required_fields)
merged_df = convert_json_to_csv(merged_json, merged_csv, required_fields)

INFO:__main__:Converted D:\A_CSE499\data\eduqg_903.json to D:\A_CSE499\data\eduqg_903.csv
INFO:__main__:Converted D:\A_CSE499\data\merged_600_per_level.json to D:\A_CSE499\data\merged_600_per_level.csv


In [9]:
# Punctuation cleaning function
def fix_punctuation_spacing(text):
    if not isinstance(text, str):
        return text
    text = text.replace(r'\newline', ' ').replace('\n', ' ')
    text = text.replace(r'\"', '"')
    text = re.sub(r'\s+([,.;:!?])', r'\1', text)
    text = re.sub(r'([,.;:!?])([^\s\W])', r'\1 \2', text)
    text = re.sub(r'([.;:!?])(")', r'\2\1', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'"\s+', '"', text)
    text = re.sub(r'\s+"', '"', text)
    text = re.sub(r"'\s+", "'", text)
    text = re.sub(r"\s+'", "'", text)
    text = re.sub(r'\s*[-–—]+\s*', ' — ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [10]:
# Load tokenizer and add special tokens
tokenizer = AutoTokenizer.from_pretrained("D:\\A_CSE499\\outputLarge_B_phase2\\final_model\\reload_model")
special_tokens = list(BLOOM_LEVELS.values())
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Preprocessing function
def preprocess_function(batch):
    inputs = []
    targets = []
    for c, q, level in zip(batch['context'], batch['question'], batch['level']):
        c_clean = fix_punctuation_spacing(str(c)) if c else ""
        q_clean = fix_punctuation_spacing(str(q)) if q else ""
        if q_clean.startswith("What is the"):
            q_clean = q_clean.replace("What is the", "What can you tell about")
        bloom_token = BLOOM_LEVELS.get(level.lower(), '<REM>')
        inputs.append(f"{bloom_token} {c_clean}")
        targets.append(f"{bloom_token} {q_clean}")
    model_inputs = tokenizer(inputs, truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, truncation=True, max_length=64)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# Utility to validate CSV
def validate_csv(file_path, required_columns):
    if not os.path.exists(file_path):
        logger.error(f"CSV file not found: {file_path}")
        raise FileNotFoundError(f"CSV file not found: {file_path}")
    df = pd.read_csv(file_path)
    for col in required_columns:
        if col not in df.columns:
            logger.error(f"Column '{col}' missing in {file_path}")
            raise ValueError(f"Column '{col}' missing in {file_path}")
    return df

In [13]:
# Load and merge datasets
eduqg_df = validate_csv(os.path.join(DATA_DIR, "eduqg_903.csv"), ['context', 'question', 'level'])
merged_df = validate_csv(os.path.join(DATA_DIR, "merged_600_per_level.csv"), ['context', 'question', 'level'])
df = pd.concat([eduqg_df, merged_df], ignore_index=True)

# Normalize level and filter
df['level'] = df['level'].str.lower()
df = df[df['level'].isin(BLOOM_LEVELS.keys())]

# Print Bloom level distribution
bloom_counts = df['level'].value_counts()
print("Bloom level distribution in full dataset:")
print(bloom_counts)

Bloom level distribution in full dataset:
level
applying         729
understanding    716
analyzing        634
remembering      623
Name: count, dtype: int64


In [14]:
# Split into train (80%), temp (20%) with stratification
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df['level'])

# Split temp into validation (10%) and test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df['level'])

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")
print("Bloom level distribution in train set:")
print(train_df['level'].value_counts())

Train: 2161, Validation: 270, Test: 271
Bloom level distribution in train set:
level
applying         583
understanding    573
analyzing        507
remembering      498
Name: count, dtype: int64


In [15]:
# Display sample data
sample = df.sample(5)
for i, row in sample.iterrows():
    print(f"Context: {row['context'][:300]}")
    print(f"Question: {row['question']}")
    print(f"Level: {row['level']}")
    print("="*80)

Context: The demographics of the British Isles today are characterised by a generally high density of population in England, which accounts for almost 80% of the total population of the islands. In elsewhere on Great Britain and on Ireland, high density of population is limited to areas around, or close to, 
Question: What percentage of the population does England have compared to the other areas in the British Isles?
Level: analyzing
Context: Many years ago my student asked me the question, "Mrs. Kindred, why do you teach?" Without taking time to reflect, I answered, "Because someday I might say something that might make a difference in someone's life." Even though I was sincere, that wasn't a very good answer and my student didn't let i
Question: Why did the student continue to ask the question about the writer's being a teacher?
Level: applying
Context: " yes , " said the king 's daughter , " indeed i will be careful , and not touch the least thing , just as you have told me . " but

In [16]:
# Convert to HuggingFace Datasets
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)
dataset_test = Dataset.from_pandas(test_df)

# Tokenize datasets
processed_train_dataset = dataset_train.map(preprocess_function, batched=True, batch_size=50)
processed_val_dataset = dataset_val.map(preprocess_function, batched=True, batch_size=50)
processed_test_dataset = dataset_test.map(preprocess_function, batched=True, batch_size=50)

Map:   0%|          | 0/2161 [00:00<?, ? examples/s]

C:\Users\User\anaconda3\envs\torch121\Lib\site-packages\transformers\tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

In [17]:
# Evaluation Metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    labels[labels == -100] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    def remove_bloom_token(text):
        for token in BLOOM_LEVELS.values():
            text = text.replace(token, '').strip()
        return text

    decoded_preds_clean = [remove_bloom_token(pred) for pred in decoded_preds]
    decoded_labels_clean = [remove_bloom_token(ref) for ref in decoded_labels]

    ref_tokens = [[nltk.word_tokenize(ref)] for ref in decoded_labels_clean]
    pred_tokens = [nltk.word_tokenize(pred) for pred in decoded_preds_clean]

    smoothie = SmoothingFunction().method4
    bleu1 = corpus_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu4 = corpus_bleu(ref_tokens, pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = sum(
        scorer.score(ref, pred)['rougeL'].fmeasure
        for ref, pred in zip(decoded_labels_clean, decoded_preds_clean)
    ) / len(decoded_labels_clean)

    meteor = sum(
        meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred))
        for ref, pred in zip(decoded_labels_clean, decoded_preds_clean)
    ) / len(decoded_labels_clean)

    try:
        P, R, F1 = score(decoded_preds_clean, decoded_labels_clean, lang="en", verbose=False)
        bertscore = F1.mean().item()
    except Exception:
        bertscore = 0.0

    return {
        "bleu-1": bleu1, "bleu-4": bleu4,
        "rouge-l": rouge_l, "meteor": meteor, "bertscore": bertscore
    }

In [18]:
# Early stopping callback
class CustomEarlyStoppingCallback(EarlyStoppingCallback):
    def __init__(self, early_stopping_patience, min_delta=0.005):
        super().__init__(early_stopping_patience=early_stopping_patience)
        self.min_delta = min_delta
        self.best_metric = float('inf')
        self.early_stopping_patience_counter = 0

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        eval_loss = metrics.get('eval_loss', float('inf'))
        if self.best_metric == float('inf') or eval_loss < self.best_metric - self.min_delta:
            self.best_metric = eval_loss
            self.early_stopping_patience_counter = 0
        else:
            self.early_stopping_patience_counter += 1
        if self.early_stopping_patience_counter >= self.early_stopping_patience:
            logger.info(f"Early stopping triggered after {self.early_stopping_patience} evaluations with eval_loss={eval_loss}")
            control.should_training_stop = True

In [19]:
# Save trial results helper
def save_trial_results(study, output_dir):
    trials_data = []
    for trial in study.trials:
        trial_data = {
            'trial_number': trial.number,
            'eval_loss': trial.value if trial.value is not None else float('inf'),
            'state': str(trial.state),
            **trial.params
        }
        trials_data.append(trial_data)
    best_trial_data = {
        'trial_number': study.best_trial.number,
        'eval_loss': study.best_trial.value,
        'state': 'BEST',
        **study.best_params
    }
    trials_data.append(best_trial_data)
    output_path = os.path.join(output_dir, 'optuna_trials.csv')
    trials_df = pd.DataFrame(trials_data)
    mode = 'a' if os.path.exists(output_path) else 'w'
    trials_df.to_csv(output_path, index=False, mode=mode, header=not os.path.exists(output_path))
    logger.info("Saved trial results to optuna_trials.csv")

In [20]:
# Objective function for Optuna tuning
def objective(trial):
    torch.cuda.empty_cache()
    gc.collect()
    model = BartForConditionalGeneration.from_pretrained("D:\\A_CSE499\\outputLarge_B_phase2\\final_model\\reload_model", trust_remote_code=True, use_safetensors=True)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.gradient_checkpointing_enable()
    model.generation_config.no_repeat_ngram_size = 3
    model.generation_config.min_length = 5

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1, log=True)
    warmup_steps = trial.suggest_int("warmup_steps", 100, 500, step=50)
    lr_scheduler_type = trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "cosine_with_restarts"])

    trial_output_dir = os.path.join(OUTPUT_DIR, f"trial_{trial.number}")
    os.makedirs(trial_output_dir, exist_ok=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir=trial_output_dir,
        num_train_epochs=8,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=16,
        dataloader_num_workers=0,
        dataloader_pin_memory=torch.cuda.is_available(),
        lr_scheduler_type=lr_scheduler_type,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        remove_unused_columns=True,
        report_to=[],
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        weight_decay=weight_decay,
        fp16=True,
        logging_strategy="epoch",
        predict_with_generate=True,
        generation_max_length=64,
        generation_num_beams=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        group_by_length=True
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_train_dataset,
        eval_dataset=processed_val_dataset,
        compute_metrics=None,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True),
        tokenizer=tokenizer,
        callbacks=[CustomEarlyStoppingCallback(early_stopping_patience=2)]
    )

    checkpoint = None
    try:
        for checkpoint_dir in sorted(os.listdir(trial_output_dir), reverse=True):
            if checkpoint_dir.startswith("checkpoint"):
                checkpoint = os.path.join(trial_output_dir, checkpoint_dir)
                break
    except FileNotFoundError:
        logger.info(f"No checkpoints found in {trial_output_dir}")

    if checkpoint:
        try:
            logger.info(f"Resuming training from {checkpoint}")
            trainer.train(resume_from_checkpoint=checkpoint)
        except Exception as e:
            logger.warning(f"Failed to resume from checkpoint {checkpoint}: {e}. Starting new training.")
            trainer.train()
    else:
        logger.info(f"Starting new training for trial {trial.number}")
        trainer.train()

    eval_results = trainer.evaluate()
    torch.cuda.empty_cache()

    for log in trainer.state.log_history:
        if 'eval_loss' in log:
            step = log.get("epoch", 0)
            trial.report(log['eval_loss'], step=int(step))

    return eval_results["eval_loss"]

In [25]:
# Run Optuna hyperparameter search
study_name = "bart_bloom_question_generation"
storage = f"sqlite:///{os.path.join(OUTPUT_DIR, 'optuna_study.db')}"
try:
    study = optuna.create_study(
        study_name=study_name,
        storage=storage,
        direction="minimize",
        sampler=TPESampler(seed=42),  # Bayesian optimization
        pruner=MedianPruner(n_warmup_steps=2),
        load_if_exists=True
    )
    study.optimize(objective, n_trials=13)
    save_trial_results(study, OUTPUT_DIR)
    best_params = study.best_params
    with open(os.path.join(OUTPUT_DIR, "best_params.json"), "w") as f:
        json.dump(best_params, f, indent=4)
    print("Best hyperparameters:", best_params)
    print("Best objective value (eval_loss):", study.best_value)
except Exception as e:
    logger.error(f"Optuna optimization or file saving failed: {e}")
    raise

[I 2025-07-25 20:03:43,798] A new study created in RDB with name: bart_bloom_question_generation
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
C:\Users\User\AppData\Local\Temp\ipykernel_11476\692185169.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
INFO:__main__:Starting new training for trial 0


Epoch,Training Loss,Validation Loss
1,3.088900,2.417607
2,2.316900,2.183061
3,2.143000,2.158609
4,2.012300,2.145115
5,1.869500,2.148754
6,1.703100,2.150242


INFO:__main__:Early stopping triggered after 2 evaluations with eval_loss=2.1502420902252197
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


INFO:__main__:Early stopping triggered after 2 evaluations with eval_loss=2.1502861976623535
C:\Users\User\anaconda3\envs\torch121\Lib\site-packages\optuna\trial\_trial.py:501: UserWarning: The reported value is ignored because this `step` 6 is already reported.
  warnings.warn(
[I 2025-07-25 20:44:51,753] Trial 0 finished with value: 2.1502861976623535 and parameters: {'learning_rate': 1.827226177606625e-05, 'weight_decay': 0.08927180304353628, 'warmup_steps': 400, 'lr_scheduler_type': 'linear'}. Best is trial 0 with value: 2.1502861976623535.
C:\Users\User\AppData\Local\Temp\ipykernel_11476\692185169.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
INFO:__main__:Starting new training for trial 1


Epoch,Training Loss,Validation Loss
1,3.188300,2.543861
2,2.380200,2.198973
3,2.189000,2.165307
4,2.077600,2.145302
5,1.960400,2.147200
6,1.832400,2.149708


...

INFO:__main__:Early stopping triggered after 2 evaluations with eval_loss=1.7933887243270874
C:\Users\User\anaconda3\envs\torch121\Lib\site-packages\optuna\trial\_trial.py:501: UserWarning: The reported value is ignored because this `step` 7 is already reported.
  warnings.warn(
[I 2025-07-26 03:17:52,363] Trial 11 finished with value: 1.7933887243270874 and parameters: {'learning_rate': 4.8625815893389155e-05, 'weight_decay': 0.030098158477190287, 'warmup_steps': 250, 'lr_scheduler_type': 'cosine_with_restarts'}. Best is trial 11 with value: 1.7933887243270874.
C:\Users\User\AppData\Local\Temp\ipykernel_11476\692185169.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
INFO:__main__:Starting new training for trial 12


Epoch,Training Loss,Validation Loss
1,2.744900,2.207433
2,2.191800,2.173019
3,1.931600,2.183483
4,1.571900,1.897065
5,1.042700,1.972192
6,0.824600,1.989672


INFO:__main__:Early stopping triggered after 2 evaluations with eval_loss=1.9896717071533203
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


INFO:__main__:Early stopping triggered after 2 evaluations with eval_loss=1.9050681591033936
C:\Users\User\anaconda3\envs\torch121\Lib\site-packages\optuna\trial\_trial.py:501: UserWarning: The reported value is ignored because this `step` 6 is already reported.
  warnings.warn(
[I 2025-07-26 03:49:56,934] Trial 12 finished with value: 1.9050681591033936 and parameters: {'learning_rate': 4.765821424574219e-05, 'weight_decay': 0.033664587127253655, 'warmup_steps': 250, 'lr_scheduler_type': 'cosine_with_restarts'}. Best is trial 11 with value: 1.7933887243270874.
INFO:__main__:Saved trial results to optuna_trials.csv


Best hyperparameters: {'learning_rate': 4.8625815893389155e-05, 'weight_decay': 0.030098158477190287, 'warmup_steps': 250, 'lr_scheduler_type': 'cosine_with_restarts'}
Best objective value (eval_loss): 1.7933887243270874


In [26]:
# Clean up non-best trials
best_trial_dir = os.path.join(OUTPUT_DIR, f"trial_{study.best_trial.number}")
for trial_dir in os.listdir(OUTPUT_DIR):
    if trial_dir.startswith("trial_") and trial_dir != os.path.basename(best_trial_dir):
        shutil.rmtree(os.path.join(OUTPUT_DIR, trial_dir))

In [27]:
# Final Training with best params
best_model_dir = os.path.join(OUTPUT_DIR, "best_model")
os.makedirs(best_model_dir, exist_ok=True)
model = BartForConditionalGeneration.from_pretrained("D:\\A_CSE499\\outputLarge_B_phase2\\final_model\\reload_model", trust_remote_code=True, use_safetensors=True)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.gradient_checkpointing_enable()
model.generation_config.no_repeat_ngram_size = 3
model.generation_config.min_length = 5

training_args = Seq2SeqTrainingArguments(
    output_dir=best_model_dir,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    dataloader_num_workers=0,
    lr_scheduler_type=best_params["lr_scheduler_type"],
    learning_rate=best_params["learning_rate"],
    warmup_steps=best_params["warmup_steps"],
    remove_unused_columns=True,
    report_to=[],
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    weight_decay=best_params["weight_decay"],
    fp16=True,
    logging_strategy="epoch",
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    group_by_length=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_val_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True),
    tokenizer=tokenizer,
    callbacks=[CustomEarlyStoppingCallback(early_stopping_patience=3)]
)

checkpoint = None
try:
    for checkpoint_dir in sorted(os.listdir(best_model_dir), reverse=True):
        if checkpoint_dir.startswith("checkpoint"):
            checkpoint = os.path.join(best_model_dir, checkpoint_dir)
            break
except FileNotFoundError:
    logger.info(f"No checkpoints found in {best_model_dir}")

if checkpoint:
    try:
        logger.info(f"Resuming final training from {checkpoint}")
        trainer.train(resume_from_checkpoint=checkpoint)
    except Exception as e:
        logger.warning(f"Failed to resume from checkpoint {checkpoint}: {e}. Starting new training.")
        trainer.train()
else:
    logger.info("Starting new final training")
    trainer.train()

C:\Users\User\AppData\Local\Temp\ipykernel_11476\617955923.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
INFO:__main__:Starting new final training


Epoch,Training Loss,Validation Loss,Bleu-1,Bleu-4,Rouge-l,Meteor,Bertscore
1,2.743400,2.207138,0.613560,0.197788,0.141655,0.580153,0.855882
2,2.175500,2.182361,0.613555,0.184598,0.132780,0.582881,0.856541
3,1.920600,2.149946,0.613435,0.188020,0.135964,0.579550,0.856892
4,1.541400,1.990693,0.454158,0.164250,0.159382,0.447152,0.860341
5,1.037100,1.971221,0.424009,0.162269,0.164989,0.429214,0.864315
6,0.829300,2.172569,0.577612,0.236932,0.181978,0.536924,0.865397
7,0.694100,1.966817,0.601289,0.219109,0.157713,0.598908,0.858600
8,0.577600,2.132075,0.603524,0.217710,0.152872,0.600486,0.856471


INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel wer

In [28]:
# Save final model and tokenizer
reload_model_dir = os.path.join(best_model_dir, "reload_model")
os.makedirs(reload_model_dir, exist_ok=True)
try:
    model.save_pretrained(reload_model_dir, safe_serialization=True)
    tokenizer.save_pretrained(reload_model_dir)
    necessary_files = [
        "model.safetensors",
        "config.json",
        "tokenizer_config.json",
        "vocab.json",
        "merges.txt",
        "special_tokens_map.json"
    ]
    for file in os.listdir(reload_model_dir):
        if file not in necessary_files:
            os.remove(os.path.join(reload_model_dir, file))
    logger.info(f"Saved model and tokenizer to {reload_model_dir}")
except Exception as e:
    logger.error(f"Failed to save model/tokenizer to {reload_model_dir}: {e}")
    raise

INFO:__main__:Saved model and tokenizer to D:\A_CSE499\output_bloom\best_model\reload_model


In [29]:
# Evaluate on test set and save results
model.eval()
torch.cuda.empty_cache()
test_results = trainer.predict(processed_test_dataset)
test_metrics = test_results.metrics
test_results_df = pd.DataFrame([test_metrics])
test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
try:
    test_results_df.to_csv(test_results_path, index=False)
    logger.info(f"Saved test results to {test_results_path}")
except Exception as e:
    logger.error(f"Failed to save test results: {e}")
    raise

test_preds = test_results.predictions[0] if isinstance(test_results.predictions, tuple) else test_results.predictions
test_preds = np.clip(test_preds, 0, tokenizer.vocab_size - 1)
decoded_preds = tokenizer.batch_decode(test_preds, skip_special_tokens=False)
label_ids = np.clip(test_results.label_ids, 0, tokenizer.vocab_size - 1)
decoded_refs = tokenizer.batch_decode(label_ids, skip_special_tokens=False)
pred_ref_df = pd.DataFrame({
    "context": processed_test_dataset["context"],
    "level": processed_test_dataset["level"],
    "predicted_question": decoded_preds,
    "reference_question": decoded_refs
})
pred_ref_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
try:
    pred_ref_df.to_csv(pred_ref_path, index=False)
    logger.info(f"Saved test predictions to {pred_ref_path}")
except Exception as e:
    logger.error(f"Failed to save test predictions: {e}")
    raise

INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Saved test results to D:\A_CSE499\output_bloom\test_results.csv
INFO:__main__:Saved test predictions to D:\A_CSE499\output_bloom\test_predictions.csv


In [30]:
# Compute metrics per Bloom level
metrics_by_level = {}
for level, group in pred_ref_df.groupby('level'):
    preds = [p.replace(BLOOM_LEVELS.get(level.lower(), '<REM>'), '').strip() for p in group['predicted_question']]
    refs = [r.replace(BLOOM_LEVELS.get(level.lower(), '<REM>'), '').strip() for r in group['reference_question']]
    ref_tokens = [[nltk.word_tokenize(ref)] for ref in refs]
    pred_tokens = [nltk.word_tokenize(pred) for pred in preds]
    smoothie = SmoothingFunction().method4

    bleu1 = corpus_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu4 = corpus_bleu(ref_tokens, pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = sum(scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(refs, preds)) / len(refs)
    meteor = sum(meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred)) for ref, pred in zip(refs, preds)) / len(refs)
    try:
        P, R, F1 = score(preds, refs, lang="en", verbose=False)
        bertscore = F1.mean().item()
    except Exception:
        bertscore = 0.0

    metrics_by_level[level] = {
        'bleu-1': bleu1,
        'bleu-4': bleu4,
        'rouge-l': rouge_l,
        'meteor': meteor,
        'bertscore': bertscore,
        'num_examples': len(group)
    }

metrics_df = pd.DataFrame(metrics_by_level).T
metrics_path = os.path.join(OUTPUT_DIR, "metrics_by_level.csv")
metrics_df.to_csv(metrics_path)
logger.info(f"Saved per-level metrics to {metrics_path}")
print("Metrics by Bloom level:")
print(metrics_df)


INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Using default tokenizer.
Some weights of RobertaModel wer

Metrics by Bloom level:
                 bleu-1    bleu-4   rouge-l    meteor  bertscore  num_examples
analyzing      0.554460  0.267330  0.233436  0.587353   0.884223          64.0
applying       0.544616  0.267717  0.244837  0.583565   0.884355          73.0
remembering    0.538796  0.277847  0.245975  0.576238   0.888526          62.0
understanding  0.541083  0.284006  0.256050  0.585837   0.887610          72.0


In [31]:
# Display example predictions
try:
    num_examples = min(5, len(decoded_preds), len(decoded_refs), len(processed_test_dataset['context']))
    logger.info("\nExample Predictions and References:")
    for i in range(num_examples):
        logger.info(f"\nExample {i+1}:")
        logger.info(f"Context: {processed_test_dataset['context'][i]}")
        logger.info(f"Level: {processed_test_dataset['level'][i]}")
        logger.info(f"Prediction: {decoded_preds[i]}")
        logger.info(f"Reference: {decoded_refs[i]}")
except Exception as e:
    logger.error(f"Failed to print example predictions: {e}")

INFO:__main__:
Example Predictions and References:
INFO:__main__:
Example 1:
INFO:__main__:Context: The Puritans   get a bad reputation in America--especially when it comes to alcohol.
Mayflower, the first ship that came over from England to Massachusetts Bay, actually carried more beer than water.In fact the Founding Fathers of America liked a drink--Samuel Adams was a partner in his father's brewery, and Thomas Jefferson was famous for importing European wines.
Early Americans took a healthful small drink for breakfast, whiskey was a typical lunchtime drink, ale   accompanied supper and the day ended with another drink called nightcap.Most Americans in 1790 consumed an average of 5.8 gallons of pure alcohol a year.In 1830, consumption reached 7.1 gallons a year and alcoholism was starting to have a serious influence on communities.Women and children might be in physical danger if the man of the house began drinking.If he became ill or lost his job through drinking, there was no socia